In [32]:
import math as m
import numpy as np

from qiskit import QuantumCircuit, transpile
from qiskit.providers.aer import QasmSimulator
from qiskit.visualization import plot_histogram
from qiskit.quantum_info import Statevector

def changePhase(circuit, qubits: []): 
    for i in range(len(qubits)):
        if(qubits[i] > circuit.width() / 2):
            raise Exception('The qubit position limit is ' + str(int((circuit.width() / 2) - 1)) + '.')
    _circuit = circuit.copy()
    # Gives phase 0 to 180 for the qubits
    for i in range(len(qubits)):
        _circuit.z(qubits[i])
    return _circuit
        
def buildWState(circuit, referencePosition: int, n: int):
    if (n < 2):
        raise Exception('The size must be at least two.')
    if (referencePosition < 0):
        raise Exception('The reference position must be at least zero.')
    if (n > circuit.width() / 2):
        raise Exception('The circuit does not have enough qubits.')
    if (referencePosition + n > circuit.width() / 2):
        raise Exception('The reference position must be lower.')
    firstPosition = referencePosition
    lastPosition = referencePosition + n - 1
    _circuit = circuit.copy()
    for i in range(firstPosition, lastPosition + 1):
        _circuit.reset(i)
    if (n >= 3):
        _circuit.ry(2 * np.arccos(1 / m.sqrt(n)), firstPosition)
        for i in range(n - 3):
            _circuit.cry(2 * np.arccos(1 / m.sqrt(n - (i + 1))), firstPosition + i, firstPosition + i + 1)
    if (n == 2):
        _circuit.h(firstPosition)
    else:
        _circuit.ch(lastPosition - 2, lastPosition - 1)
    for i in reversed(range(firstPosition, lastPosition)):
        _circuit.cx(i, i + 1)
    _circuit.x(firstPosition)
    
    return _circuit

# Use Aer's qasm_simulator
simulator = QasmSimulator()

# Create a Quantum Circuit acting on the q register
q = 5
circuit = QuantumCircuit(q,q)
circuit = buildWState(circuit, 0, 3)
circuit.barrier()
circuit = changePhase(circuit, [0,1,8])

psi = Statevector.from_instruction(circuit)
# Probabilities for measuring qubits
probs = psi.probabilities_dict()
print('probs: {}'.format(probs))

# Compile the circuit down to low-level QASM instructions
# supported by the backend
compiled_circuit = transpile(circuit,simulator)

# Execute the circuit on the qasm simulator
job = simulator.run(compiled_circuit, shots=1000)

#Grab results from the job
results = job.result()

# Draw the circuit
circuit.draw()

Exception: The qubit position limit is 4.